In [ ]:
# Importing necessary libraries
import os  
import re  
import cv2  
import random  
import numpy as np  
import pandas as pd  
import tensorflow as tf 

# Setting the path to the directory containing preprocessed images
DATA_PATH = "/kaggle/input/ocular-disease-recognition-odir5k/preprocessed_images"

# Defining the target image size for resizing images
IMG_SIZE = 224

# Loading the dataset from a CSV file into a pandas DataFrame
data = pd.read_csv("/kaggle/input/ocular-disease-recognition-odir5k/full_df.csv")


In [ ]:
# Mapping of short class labels to their full descriptive names
class_short2full = {
    "G": "Glaucoma",  # Short label 'G' represents Glaucoma
    "C": "Cataract",  # Short label 'C' represents Cataract
    "A": "Age Related Macular Degeneration",  # Short label 'A' represents ARMD
    "H": "Hypertension",  # Short label 'H' represents Hypertension
    "M": "Myopia"  # Short label 'M' represents Myopia
}

# Mapping of short class labels to numerical indices for machine learning models
class_dict = {
    "G": 0,  # Glaucoma is assigned index 0
    "C": 1,  # Cataract is assigned index 1
    "A": 2,  # ARMD is assigned index 2
    "H": 3,  # Hypertension is assigned index 3
    "M": 4   # Myopia is assigned index 4
}


In [ ]:
# Data preprocessing and converting class labels
data["class"] = data["labels"].apply(lambda x: " ".join(re.findall("[a-zA-Z]+", x)))

CLASSES = ["G", "C", "A", "H", "M"]

In [ ]:
# Create a dictionary mapping each class to a list of image filenames
# For each class in class_short2full.keys(), filter the dataset where "class" matches the class label
# Extract the "filename" column values and store them as a NumPy array in the dictionary

dict_img_list = {
    class_: data.loc[data["class"] == class_]["filename"].values  # Extract filenames for each class
    for class_ in class_short2full.keys()  # Iterate over all defined classes
}


In [ ]:
# Function to create a dataset by processing images from the given list
# Supports augmentations (e.g., flipping, rotation) if specified

def create_dataset(img_list, class_label, max_images, augment={}):
    dataset = []  # Initialize an empty list to store processed images and their labels
    count = 0  # Counter to track the number of processed images

    # Loop through each image in the provided list
    for img in img_list:
        # Stop processing if the max_images limit is reached
        if max_images is not None and count >= max_images:
            break

        # Construct the full image path and read the image
        image_path = os.path.join(DATA_PATH, img)
        image = cv2.imread(image_path)

        # Skip if the image couldn't be loaded
        if image is None:
            continue

        # Convert the image to RGB format and resize to the target size
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))

        # Perform augmentations if the class label is in the augment dictionary
        if augment and random.random() < 1:  # Ensure augmentation logic runs
            for _ in range(3):  # Generate three augmented versions of the image
                if class_label in augment.keys():
                    # Apply left-right flip
                    image_lr = tf.image.flip_left_right(image)
                    # Apply up-down flip
                    image_ud = tf.image.flip_up_down(image)
                    # Apply 90-degree rotation
                    image_rot90 = tf.image.rot90(image, k=1)

                    # Append augmented images with their labels to the dataset
                    dataset.append([np.array(image_lr), class_label])
                    dataset.append([np.array(image_ud), class_label])
                    dataset.append([np.array(image_rot90), class_label])

                    count += 3  # Increment the counter for each augmentation

        # Append the original image with its label to the dataset
        dataset.append([np.array(image), class_label])
        count += 1  # Increment the counter for the original image

    # Shuffle the dataset to randomize the order of images
    random.shuffle(dataset)

    return dataset  # Return the prepared dataset


In [ ]:
# Dataset preparation for different ocular disease classes, with augmentations and random shuffling
# Initialize lists to store the datasets for each class

dataset_G = []  
augment_G = {  # Set augmentation flag for Glaucoma class (class 0)
    class_dict["G"]: True,
}

# Loop through each class for Glaucoma (CLASSES[0]) and generate the dataset
for i, class_ in enumerate(CLASSES[0]):
    img_list = dict_img_list[class_]  # Get the list of images for the current class
    dataset_G += create_dataset(img_list, 0, 284, augment_G)  # Create and add to the dataset

    random.shuffle(dataset_G)  # Shuffle the dataset to randomize the order

###### 
dataset_C = []  
augment_C = {  # Set augmentation flag for Cataract class (class 1)
    class_dict["C"]: True,
}

# Loop through each class for Cataract (CLASSES[1]) and generate the dataset
for i, class_ in enumerate(CLASSES[1]):
    img_list = dict_img_list[class_]
    dataset_C += create_dataset(img_list, 1, 293, augment_C)

    random.shuffle(dataset_C)

###########
dataset_A = []  
augment_A = {  # Set augmentation flag for Age Related Macular Degeneration class (class 2)
    class_dict["A"]: True,
}

# Loop through each class for Age Related Macular Degeneration (CLASSES[2]) and generate the dataset
for i, class_ in enumerate(CLASSES[2]):
    img_list = dict_img_list[class_]
    dataset_A += create_dataset(img_list, 2, 266, augment_A)

    random.shuffle(dataset_A)

###########
dataset_H = []  
augment_H = {  # Set augmentation flag for Hypertension class (class 3)
    class_dict["H"]: True,
}

# Loop through each class for Hypertension (CLASSES[3]) and generate the dataset
for i, class_ in enumerate(CLASSES[3]):
    img_list = dict_img_list[class_]
    dataset_H += create_dataset(img_list, 3, 128, augment_H)

    random.shuffle(dataset_H)
###########
dataset_M = []  
augment_M = {  # Set augmentation flag for Myopia class (class 4)
    class_dict["M"]: True,
}

# Loop through each class for Myopia (CLASSES[4]) and generate the dataset
for i, class_ in enumerate(CLASSES[4]):
    img_list = dict_img_list[class_]
    dataset_M += create_dataset(img_list, 4, 232, augment_M)

    random.shuffle(dataset_M)


In [ ]:
H=len(dataset_H)
M=len(dataset_M)
A=len(dataset_A)
G=len(dataset_G)
C=len(dataset_C)

In [ ]:
# Combine datasets from all classes (Glaucoma, Hypertension, Age Related Macular Degeneration, Myopia, Cataract)
dataset = []
dataset += dataset_G + dataset_H + dataset_A + dataset_M + dataset_C

# Get the total number of samples in the combined dataset
len(dataset)


In [ ]:
import matplotlib.pyplot as plt

# Choose a random image
random_index = random.randint(0, len(dataset) - 1)
image, class_label = dataset[random_index]

# View image
plt.imshow(image)
plt.title(f"Class: {class_label}")
plt.axis('off')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Parameters
image_size = 224
num_classes = 5
train_ratio = 0.7
val_ratio = 0.15

# Preparing predictors and target variables
train_x = np.array([i[0] for i in dataset]).reshape(-1, image_size, image_size, 3)
train_y = np.array([i[1] for i in dataset])

# Calculating the number of images for each split
num_images = len(train_x)
num_train = int(num_images * train_ratio)
num_val = int(num_images * val_ratio)
num_test = num_images - num_train - num_val

# Splitting the dataset into train and remaining (validation + test)
x_train, x_remaining, y_train, y_remaining = train_test_split(train_x, train_y, train_size=num_train, random_state=42)

# Further splitting the remaining data into validation and test
x_val, x_test, y_val, y_test = train_test_split(x_remaining, y_remaining, test_size=num_test, random_state=42)

# Convert labels to categorical
y_train = to_categorical(y_train, num_classes)
y_val = to_categorical(y_val, num_classes)
y_test = to_categorical(y_test, num_classes)

# Print the number of images in each split
print(f"Number of images - Train: {len(x_train)}, Validation: {len(x_val)}, Test: {len(x_test)}")


In [ ]:
print("Shape of x_train:", x_train.shape)
print("Shape of x_val:", x_val.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of x_test:", x_test.shape)
print("Shape of y_test:", y_test.shape)

In [ ]:
from tensorflow.keras.utils import to_categorical

# Convert the target labels into categorical (one-hot encoded) format for each dataset split
y_train = to_categorical(y_train, num_classes=num_classes)  # One-hot encoding for training labels
y_val = to_categorical(y_val, num_classes=num_classes)  # One-hot encoding for validation labels
y_test = to_categorical(y_test, num_classes=num_classes)  # One-hot encoding for test labels


In [ ]:
print("Shape of y_train:", y_train.shape)
print("Shape of y_val:", y_val.shape)
print("Shape of y_test:", y_test.shape)
print("Shape of x_train:", x_train.shape)
print("Shape of x_val:", x_val.shape)
print("Shape of x_test:", x_test.shape)

In [ ]:
# Convert one-hot encoded labels to class labels
y_train_labels = np.argmax(y_train, axis=1)

# Count the occurrences of each class in the test set
test_class_counts = np.bincount(y_train_labels)

# Print the number of images for each class in the test set
for class_label, count in enumerate(test_class_counts):
    print(f"Class {class_label}: {count} images")

In [ ]:
# Save the combined training data for future use
np.save('/kaggle/working/x_train_0-5.npy', x_train)
np.save('/kaggle/working/y_train_0-5.npy', y_train)

np.save('/kaggle/working/x_val_0-5.npy', x_val)
np.save('/kaggle/working/y_val_0-5.npy', y_val)

np.save('/kaggle/working/x_test_0-5.npy', x_test)
np.save('/kaggle/working/y_test_0-5.npy', y_test)

In [ ]:
print( y_train.shape, type(y_train))